<a href="https://colab.research.google.com/github/ZYF-B/Pytorch_learning/blob/main/CNN_Res.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
%matplotlib inline


torch.manual_seed(1024)

In [ ]:
dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
train_set, val_set = random_split(dataset, [50000, 10000])
test_set = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())
len(train_set), len(val_set), len(test_set)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 12693404.72it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 345231.01it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 3200875.39it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4283905.73it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



(50000, 10000, 10000)

In [ ]:
train_loader = DataLoader(train_set, batch_size=500, shuffle=True)
val_loader = DataLoader(val_set, batch_size=500, shuffle=True)
test_loader = DataLoader(test_set, batch_size=500, shuffle=True)

In [ ]:
eval_iters = 10

def estimate_loss(model):
    re = {}
    model.eval()
    re['train'] = _loss(model, train_loader)
    re['val'] = _loss(model, val_loader)
    re['test'] = _loss(model, test_loader)
    model.train()
    return re


@torch.no_grad()
def _loss(model, dataloader):
    loss = []
    acc = []
    data_iter = iter(dataloader)
    for t in range(eval_iters):
        inputs, labels = next(data_iter)
        # inputs: (B, 1, 28, 28)
        # labels: (B)
        B, C, H, W = inputs.shape
        logits = model(inputs)
        loss.append(F.cross_entropy(logits, labels))
        preds = torch.argmax(logits, dim=-1)
        acc.append((preds == labels).sum() / B)
    re = {
        'loss': torch.tensor(loss).mean().item(),
        'acc': torch.tensor(acc).mean().item()
    }
    return re

In [ ]:
def train(model, optimizer, epochs=10):
    lossi = []
    for e in range(epochs):
        for data in train_loader:
            inputs, labels = data
            logits = model(inputs)
            loss = F.cross_entropy(logits, labels)
            lossi.append(loss.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        stats = estimate_loss(model)
        train_loss = f'{stats["train"]["loss"]:.3f}'
        val_loss = f'{stats["val"]["loss"]:.3f}'
        test_loss = f'{stats["test"]["loss"]:.3f}'
        print(f'epoch {e} train {train_loss} val {val_loss} test {test_loss}')
    return lossi

In [ ]:
class Conv2d(nn.Module):
  #代码逻辑没问题，但效率很低

  def __init__(self, in_channel, out_channel, kernel_size, stride=1, padding=0):
    super().__init__()
    self.stride = stride
    self.padding = padding
    self.out_channel = out_channel
    self.kernel_size = kernel_size
    self.weight = nn.Parameter(torch.randn((out_channel, in_channel) + kernel_size))
    self.bias = nn.Parameter(torch.randn(out_channel))

  def forward(self, x):
    data = x.unsqueeze(0) if len(x.shape) == 3 else x
    B, I, H, W = data.shape
    h_step = (H + 2 * self.padding - self.kernel_size[0]) // self.stride + 1
    w_step = (W + 2 * self.padding - self.kernel_size[1]) // self.stride + 1
    output = torch.zeros(B, self.out_channel, h_step, w_step)
    # 在图像的边缘增加0
    data = F.pad(data, (self.padding,) * 4)
    for i in range(h_step):
      h_begin = i * self.stride
      h_end = h_begin + self.kernel_size[0]
      for j in range(w_step):
        w_being = j * self.stride
        w_end = w_being + self.kernel_size[1]
        inputs = data[:, :, h_begin: h_end, w_being: w_end].unsqueeze(1)
        # inputs：   (B,  1, I, kernel_size[0], kernel_size[1]）
        # self.weight: (out_channel, I, kernel_size[0], kernel_size[1]）
        # linear_out: (B, out_channel）
        # bias：    (   out_channel）
        linear_out = (inputs * self.weight).sum((-3, -2, -1))
        output[:, :, i, j] = linear_out + self.bias
    return output.squeeze(0) if len(x.shape) == 3 else output


In [ ]:
class ResidualBlock(nn.Module):

    def __init__(self, in_channels, out_channels, stride=1):
      super().__init__()
      self.conv1 = nn.Conv2d(in_channels, out_channels, (3, 3), stride=stride, padding=1)
      self.bn1 = nn.BatchNorm2d(out_channels)
      self.conv2 = nn.Conv2d(out_channels, out_channels, (3, 3), stride=1, padding=1)
      self.bn2 = nn.BatchNorm2d(out_channels)
      self.downsample = None
      if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Conv2d(in_channels, out_channels, (1, 1), stride=stride, padding=0)
            self.bn3 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
      inputs = x
      x = F.relu(self.bn1(self.conv1(x)))
      x = self.bn2(self.conv2(x))
      if self.downsample is not None:
        inputs = self.bn3(self.downsample(inputs))
      out = x + inputs
      out = F.relu(out)
      return out

In [ ]:
class ResNet(nn.Module):

    def __init__(self):
        super().__init__()
        self.block1 = ResidualBlock(1, 20)
        self.block2 = ResidualBlock(20, 40, stride=2)
        self.block3 = ResidualBlock(40, 60, stride=2)
        self.block4 = ResidualBlock(60, 80, stride=2)
        self.block5 = ResidualBlock(80, 100, stride=2)
        self.block6 = ResidualBlock(100, 120, stride=2)
        self.fc = nn.Linear(120, 10)

    def forward(self, x):
        # x : (B, 1, 28, 28)
        B = x.shape[0]
        x = self.block1(x)  # (B, 20, 28, 28)
        x = self.block2(x)  # (B, 40, 14, 14)
        x = self.block3(x)  # (B, 60,  7,  7)
        x = self.block4(x)  # (B, 80,  4,  4)
        x = self.block5(x)  # (B, 100, 2,  2)
        x = self.block6(x)  # (B, 120, 1,  1)
        x = self.fc(x.view(B, -1))
        return x

In [ ]:
model = ResNet()
x = torch.randn(100, 1, 28, 28)
model(x).shape

torch.Size([100, 10])

In [ ]:
_ = train(model, optim.SGD(model.parameters(), lr=0.01), epochs=5)

epoch 0 train 0.242 val 0.269 test 0.248
epoch 1 train 0.137 val 0.160 test 0.144
epoch 2 train 0.096 val 0.115 test 0.109
epoch 3 train 0.070 val 0.103 test 0.091
epoch 4 train 0.051 val 0.090 test 0.077


In [ ]:
estimate_loss(model)

{'train': {'loss': 0.05184324458241463, 'acc': 0.9894000887870789},
 'val': {'loss': 0.08184759318828583, 'acc': 0.979200005531311},
 'test': {'loss': 0.07030671834945679, 'acc': 0.9825999140739441}}